# OrbitalElements interactive examples

## Working environment & Packages

Set the working environment to the repository root.

In [ ]:
using Plots;
# Using PlotlyJS as backend for plots for interactivity.
# Might fail in some notebook environments.
# Comment and use default backend if it does.
plotlyjs();

In [ ]:
using Pkg
Pkg.activate(@__DIR__()*"/../.")
import OrbitalElements as OE

# Generic parameters

## Potential

In [ ]:
G, M, bc = 1.0, 1.0, 1.0
ψ(r::Float64)   = OE.ψIsochrone(r,bc,M,G)
dψ(r::Float64)  = OE.dψIsochrone(r,bc,M,G)
d2ψ(r::Float64) = OE.d2ψIsochrone(r,bc,M,G)
Ω₀ = OE.Ω₀Isochrone(bc,M,G);

## Parameters

In [ ]:
# Radii for frequency truncations
rmin, rmax = 0., Inf
# Tolerances / Taylor expansions
EDGE = 0.01
TOLECC = 0.01
TOLA = 1.0
NINT = 32

params = OE.OrbitalParameters(Ω₀=Ω₀,rmin=rmin,rmax=rmax,
                                            EDGE=EDGE,TOLA=TOLA,TOLECC=TOLECC,
                                            NINT=NINT);

# Frequency computation

Frequency values along a constant eccentricity line. Example using the isochrone potential for which analytical expression of the frequencies are known. Comparing analytical and numerical results.

In [ ]:
function FrequenciesE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (2,length(tabe))
    for j=1:ne
        Ω1, Ω2 = OE.ComputeFrequenciesAE(ψ,dψ,d2ψ,a,tabe[j],params)
        @inbounds results[1,j], results[2,j] = Ω1, Ω2
    end
end;
function IsochroneFrequenciesE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (2,length(tabe))
    for j=1:ne
        Ω1true, Ω2true = OE.IsochroneOmega12FromAE(a,tabe[j])
        @inbounds results[1,j], results[2,j] = Ω1true, Ω2true
    end
end;

In [ ]:
ne = 200;
tabe = collect(LinRange(0.,1.,ne));
tabΩ = zeros(2,ne);
tabΩIso = zeros(2,ne);

In [ ]:
a = 1.0

# Compute
FrequenciesE!(a,tabe,tabΩ)
IsochroneFrequenciesE!(a,tabe,tabΩIso)

# Plot
xlabelΩ = ["e" "e"]
ylabelΩ = ["Ω1" "Ω2"]
p1 = plot(tabe,tabΩ[1,:],xlabel=xlabelΩ[1],ylabel=ylabelΩ[1],label="Numerical");
plot!(tabe,tabΩIso[1,:],label="True")
p2 = plot(tabe,tabΩ[2,:],xlabel=xlabelΩ[2],ylabel=ylabelΩ[2],label=false);
plot!(tabe,tabΩIso[2,:],label=false)
plot(p1,p2,layout=(1,2),size=(900,300))

# Frequency derivatives computation

Frequency values along a constant eccentricity line. Example using the isochrone potential for which analytical expression of the frequencies are known. Comparing analytical and numerical results.

In [ ]:
function FrequenciesDerivE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (6,length(tabe))
    for j=1:ne
        Ω1, Ω2, ∂Ω1∂a, ∂Ω2∂a, ∂Ω1∂e, ∂Ω2∂e = OE.ComputeFrequenciesAEWithDeriv(ψ,dψ,d2ψ,a,tabe[j],params)
        @inbounds results[1,j], results[2,j], results[3,j], results[4,j], results[5,j], results[6,j]= Ω1, Ω2, ∂Ω1∂a, ∂Ω2∂a, ∂Ω1∂e, ∂Ω2∂e
    end
end;

In [ ]:
ne = 200;
tabe = collect(LinRange(0.,1.,ne));
tabderΩ = zeros(6,ne);

In [ ]:
a = 1.0

# Compute
FrequenciesDerivE!(a,tabe,tabderΩ)

# Plot
xlabelderΩ = ["" "" "" "" "e" "e"]
ylabelderΩ = ["Ω1" "Ω2" "∂Ω1∂a" "∂Ω2∂a" "∂Ω1∂e" "∂Ω2∂e"]
plot(tabe,transpose(tabderΩ),layout=(3,2),xlabel=xlabelderΩ,ylabel=ylabelderΩ,legend=false,size=(900,600))

# Energy, angular momentum and derivatives computation

In [ ]:
function ELDerivE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (6,length(tabe))
    for j=1:ne
        E, L, ∂E∂a, ∂L∂a, ∂E∂e, ∂L∂e = OE.ComputeELAEWithDeriv(ψ,dψ,a,tabe[j],params)
        @inbounds results[1,j], results[2,j], results[3,j], results[4,j], results[5,j], results[6,j]= E, L, ∂E∂a, ∂L∂a, ∂E∂e, ∂L∂e
    end
end;

In [ ]:
ne = 200;
tabe = collect(LinRange(0.,1.,ne));
tabderEL = zeros(6,ne);

In [ ]:
a = 1.0

# Compute
ELDerivE!(a,tabe,tabderEL)

# Plot
xlabelEL = ["" "" "" "" "e" "e"]
ylabelEL = ["E" "L" "∂E∂a" "∂L∂a" "∂E∂e" "∂L∂e"]
plot(tabe,transpose(tabderEL),layout=(3,2),xlabel=xlabelEL,ylabel=ylabelEL,legend=false,size=(900,600))

# Inversions

In [ ]:
# Containers
na, ne = 100, 101;
taba = zeros(Float64,na);
tabe = zeros(Float64,ne);

tabAE = zeros(Float64,2,na*ne);
restabAE = zeros(Float64,2,na*ne);
errtabAE = zeros(Float64,na,ne);

tabInversion = zeros(Float64,2,na*ne);

In [ ]:
function InversionMapping!(FromAE::F0,ToAE::F1,amin,amax,emin,emax) where {F0 <: Function, F1 <: Function}
    
    for ka = 1:na
        a = (ka-1)*(amax-amin)/(na-1) + amin
        taba[ka] = a
    end
    for ke = 1:ne
        e = (ke-1)*(emax-emin)/(ne-1) + emin
        tabe[ke] = e
    end
    
    count = 1
    for ka = 1:na
        a = taba[ka] 
        for ke = 1:ne
            e = tabe[ke]
            
            map1, map2 = FromAE(a,e)
            aback, eback = ToAE(map1,map2)
            
            tabAE[1,count], tabAE[2,count] = a, e 
            tabInversion[1,count], tabInversion[2,count] = map1,map2
            restabAE[1,count], restabAE[2,count] = aback, eback
            errtabAE[ka,ke] = abs((aback-a)/a) + abs(eback-e)
            
            count += 1
        end
    end
end;
function InversionMappingPlot(xlabel::String,ylabel::String,perm::Bool=false)
    # Mapped space shape
    k, l = perm ? (2, 1) : (1, 2)
    p1 = scatter(tabInversion[k,:],tabInversion[l,:],xlabel=xlabel,ylabel=ylabel,legend=false,markersize=0.01)
    # (a,e)->mapped variables->(a,e) result
    p2 = scatter(restabAE[1,:],restabAE[2,:],xlabel="a",ylabel="e",legend=false,markersize=0.01)
    # Error made on (a,e)->mapped variables->(a,e) in log scale
    p3 = heatmap(taba,tabe,transpose(log10.(errtabAE)),xlabel="a",ylabel="e",legend=false)
    plot(p1,p2,p3,layout=(1,3),size=(900,300))
end;

## (a,e) &harr; (E,L) 

In [ ]:
function AEToELmapping!(amin,amax,emin,emax)
    
    fromae(a::Float64,e::Float64) = OE.ELFromAE(ψ,dψ,a,e,params)
    toae(E::Float64,L::Float64) = OE.ComputeAEFromEL(ψ,dψ,E,L,params)
    
    InversionMapping!(fromae,toae,amin,amax,emin,emax)
end;

In [ ]:
# Semi-major axis and eccentricity domains
amin, amax = 0.1, 10.0
emin, emax = 0.0, 1.0

# Compute
AEToELmapping!(amin,amax,emin,emax)

# Plot
InversionMappingPlot("E","L",true)

## (a,e) &harr; (Jr,L) 

In [ ]:
function AEToActionsmapping!(amin,amax,emin,emax)
    
    fromae(a::Float64,e::Float64) = OE.ComputeActionsAE(ψ,dψ,a,e,params)
    toae(J::Float64,L::Float64) = OE.ComputeAEFromActions(ψ,dψ,J,L,params)
    
    InversionMapping!(fromae,toae,amin,amax,emin,emax)
end;

In [ ]:
# Semi-major axis and eccentricity domains
amin, amax = 0.1, 10.0
emin, emax = 0.0, 1.0

# Compute
AEToActionsmapping!(amin,amax,emin,emax)

# Plot
InversionMappingPlot("J","L",true)

## (a,e) &harr; ($\Omega$1,$\Omega$2) 

In [ ]:
function AEToFrequenciesmapping!(amin,amax,emin,emax)
    
    fromae(a::Float64,e::Float64) = OE.ComputeFrequenciesAE(ψ,dψ,d2ψ,a,e,params)
    toae(Ω1::Float64,Ω2::Float64) = OE.ComputeAEFromFrequencies(ψ,dψ,d2ψ,Ω1,Ω2,params)
    
    InversionMapping!(fromae,toae,amin,amax,emin,emax)
end;

In [ ]:
# Semi-major axis and eccentricity domains
amin, amax = 0.1, 10.0
emin, emax = 0.0, 1.0

# Compute
AEToFrequenciesmapping!(amin,amax,emin,emax)

# Plot
InversionMappingPlot("Ω1","Ω2",true)

# Resonance variables

In [ ]:
# Containers
nu, nv = 100, 101;
tabu = collect(LinRange(-1.0,1.0,nu))
tabvp = collect(LinRange(0.,1.0,nv))
        
nα, nβ = nu, nv;
tabα = zeros(Float64,nα);
tabβ = zeros(Float64,nβ);

tabvminvmax = zeros(Float64,2,nu);
tabUV = zeros(Float64,2,nu*nv);
tabαβ = zeros(Float64,2,nα*nβ);
tabnΩ = zeros(Float64,nα,nβ);

In [ ]:
# Function to tweak the point distribution along resonance line (v∝αⁿ)
function vFromvp(vp::Float64,vmin::Float64,vmax::Float64,vmapn::Int64=2)
    return (vmax-vmin)*(vp^vmapn)+vmin
end;

## Resonances lines

In [ ]:
function reslines(n1::Int64,n2::Int64)
    
    αmin, αmax = OE.αminmax(dψ,d2ψ,params.rmin,params.rmax,params.Ω₀)
    βc(αc::Float64)::Float64 = OE.βcirc(αc,dψ,d2ψ,params)
    βmin, βmax = 0.5, βc(αmin)
    
    for kα = 1:nα
        α = (kα-1)*(αmax-αmin)/(nα-1) + αmin
        tabα[kα] = α
    end
    for kβ = 1:nβ
        β = (kβ-1)*(βmax-βmin)/(nβ-1) + βmin
        tabβ[kβ] = β
    end
    
    for kα = 1:nα
        α = tabα[kα]
        for kβ = 1:nβ
            β = tabβ[kβ]
            tabnΩ[kα,kβ] = (0.5 <= β <= βc(α)) ? n1*α + n2*α*β : Inf
        end
    end
end;

In [ ]:
# Resonance number
n1, n2 = -2, 2;

# Compute
reslines(n1,n2)

# Plot
contourf(tabα,tabβ,transpose(tabnΩ),levels=25,xlabel="α",ylabel="β",legend=false,size=(600,300))

## (u,v) &rarr; ($\alpha$,$\beta$)

In [ ]:
function UVToαβmapping(n1::Int64,n2::Int64,vmapn::Int64=2)
    
    ωmin, ωmax = OE.Findωminωmax(n1,n2,dψ,d2ψ,params)
    
    count = 1
    for ku = 1:nu
        
        u = tabu[ku]
        vmin,vmax = OE.FindVminVmax(u,n1,n2,dψ,d2ψ,ωmin,ωmax,params)
        tabvminvmax[1,ku], tabvminvmax[2,ku] = vmin, vmax
        
        for kvp = 1:nv
            vp = tabvp[kvp]
            
            v = vFromvp(vp,vmin,vmax,vmapn)
            
            α, β = OE.αβFromUV(u,v,n1,n2,ωmin,ωmax)
            
            tabUV[1,count], tabUV[2,count] = u, v 
            tabαβ[1,count], tabαβ[2,count] = α, β
            
            count += 1
        end
    end
end;

In [ ]:
# Resonance number
n1, n2 = -2, 2;
# Point distribution along resonance line (v∝αⁿ)
vmapn = 1

# Compute
UVToαβmapping(n1,n2,vmapn)

# Plot
# u,v space shape
p1 = scatter(tabUV[1,:],tabUV[2,:],markersize=0.01,xlabel="u",ylabel="v",legend=false)
# α,β space shape
p2 = scatter(tabαβ[1,:],tabαβ[2,:],markersize=0.01,xlabel="α",ylabel="β",legend=false)
plot(p1,p2,layout=(1,2),size=(900,300))

## ($\alpha$,$\beta$) &rarr; (u,v) 

In [ ]:
function αβToUVmapping(n1::Int64,n2::Int64)
    
    αmin, αmax = OE.αminmax(dψ,d2ψ,params.rmin,params.rmax,params.Ω₀)
    ωmin, ωmax = OE.Findωminωmax(n1,n2,dψ,d2ψ,params)
    βc(αc::Float64)::Float64 = OE.βcirc(αc,dψ,d2ψ,params)
    
    for kα = 1:nα
        α = (kα-1)*(αmax-αmin)/(nα-1) + αmin
        tabα[kα] = α
    end

    count = 1
    for kα = 1:nα
        α = tabα[kα]
        βmin, βmax = 0.5, βc(α)
        for kβ = 1:nβ
            β = (kβ-1)*(βmax-βmin)/(nβ-1) + βmin

            u, v = OE.UVFromαβ(α,β,n1,n2,ωmin,ωmax)
            
            tabUV[1,count], tabUV[2,count] = u, v 
            tabαβ[1,count], tabαβ[2,count] = α, β
            
            count += 1
        end
    end
end;

In [ ]:
# Resonance number
n1, n2 = -2, 2;

# Compute
αβToUVmapping(n1,n2)

# Plot
# α,β space shape
p1 = scatter(tabαβ[1,:],tabαβ[2,:],markersize=0.01,xlabel="α",ylabel="β",legend=false)
# u,v space shape
p2 = scatter(tabUV[1,:],tabUV[2,:],markersize=0.01,xlabel="u",ylabel="v",legend=false)

plot(p1,p2,layout=(1,2),size=(900,300))